In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai

!pip install llama-index

In [ ]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

tools = [multiply_tool, add_tool]

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key: ")

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
from llama_index.core.agent import AgentRunner, ReActAgentWorker, ReActAgent

In [ ]:
react_step_engine = ReActAgentWorker.from_tools(tools, llm=llm, verbose=True)
agent = AgentRunner(react_step_engine)

In [ ]:
# Start the agent interaction
print("Simple Chat Interaction:")
response = agent.chat("Hi")
print(response)

# Handle a computational query
print("\nComputational Query Interaction:")
response = agent.chat("What is (121 * 3) + 42?")
print(response)

# Task-based Execution
print("\nStep-Wise Task Execution:")
task = agent.create_task("What is (121 * 3) + 42?")

# Step 1: Execute the first step of the task
step_output = agent.run_step(task.task_id)
print(f"Step 1 Output: {step_output.output.response}")

# Step 2: Execute the next step of the task
step_output = agent.run_step(task.task_id)
print(f"Step 2 Output: {step_output.output.response}")

# Inspect the task's state
print("\nTask Details:")
tasks = agent.list_tasks()
print(f"Total Tasks: {len(tasks)}")

task_state = tasks[-1]
print(f"Task Input: {task_state.input}")

# Completed Steps
completed_steps = agent.get_completed_steps(task_state.task_id)
print(f"\nCompleted Steps ({len(completed_steps)}):")
for idx, step in enumerate(completed_steps):
    print(f"Step {idx + 1}:")
    print(f"Response: {step.output.response}")
    print(f"Sources: {step.output.sources}")